In [3]:
import traffic
import pandas as pd 
import os
from traffic.core import Traffic
from biblio_herramienta.herramienta import *
from biblio_herramienta.tratardatos import *

In [4]:
carpetaDatos = r'datos_sectores/' # carpeta que contiene los datos
nombreDatos = "datos_filtrados.csv" 
vuelos = cargardatosfiltrados(carpetaDatos,nombreDatos)
vuelos

count
icao24 callsign       
4ca60f IBK5BV     3683
4cace5 IBK6054    2974
       IBK6055    2343
49514c TAP434M    2175
344495 GES381L    2052
4cacc6 RYR3PE     2036
4853d1 TRA6845    1926
4cabae RYR4Z      1886
3c56f7 EWG4TP     1814
4ca8da RYR6BM     1805

## Cluster
Se va a intentar clasificar las trayectorias en función de su rumbo. Para eso se va utilziar el método enseñado en traffic

In [5]:
vuelos_un = vuelos.assign_id().unwrap().eval(max_workers=4)
vuelos_un.data.head()

,altitude,callsign,groundspeed,icao24,latitude,longitude,timestamp,track,flight_id,track_unwrapped
0,40000.0,RAM760R,465.0,020043,42.348953,-2.709566,2019-02-07 08:37:31+00:00,28.659632,RAM760R_011,28.659632
1,40000.0,RAM760R,465.0,020043,42.352707,-2.706757,2019-02-07 08:37:32+00:00,28.659632,RAM760R_011,28.659632
2,40000.0,RAM760R,465.0,020043,42.370450,-2.693694,2019-02-07 08:37:42+00:00,28.767649,RAM760R_011,28.767649
3,40000.0,RAM760R,465.0,020043,42.370450,-2.693694,2019-02-07 08:37:43+00:00,28.767649,RAM760R_011,28.767649
4,40000.0,RAM760R,465.0,020043,42.370450,-2.693694,2019-02-07 08:37:44+00:00,28.767649,RAM760R_011,28.767649


## Aplicación del cluster sobre los datos
La siguiente celda tiene el método de cluster aplicado sobre las trayectorias, una vez computado se genera una columna dentro de los datos que señala el cluster al que pertenece cada trayectoria.
El algoritmo usado el es el DBSCAN

## 1. DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from traffic.core.projection import CH1903

# la funcion permite aplicar algoritmos de ML a los datos que tenemos
vuelos_dbscan = vuelos_un.clustering(
    nb_samples=15, # divide en 15 puntos las trayecorias
    features=["longitude", "latitude", "track_unwrapped"], # datos cebados al algorimto de ML
    clustering=DBSCAN(eps=0.5, min_samples=10), # algoritmo de ML utilizado, con los parámetros del mismo
    transform=StandardScaler(), # lo que hace esto es escalar los datos dentro de una distribuciñ¡ón Gausiana
).fit_predict()


In [ ]:
representarcluster(vuelos_dbscan,"cluster_dbscan.png")

## 2. Optics

In [ ]:
from sklearn.cluster import OPTICS
vuelos_optics = vuelos_un.clustering(
    nb_samples = 15,
    features=["longitude", "latitude", "track_unwrapped"],
    transform = StandardScaler(),
    clustering = OPTICS(min_samples=20, eps = 0.5)
).fit_predict()

In [ ]:
representarcluster(vuelos_optics,"cluster_optics.png")

Hay que tener cuidado con los parametros de este algoritmo, sino se pone bien puede generar mas de 60 clusters 

## 3. K-medias

In [ ]:
from sklearn.cluster import KMeans
vuelos_kmeans = vuelos_un.clustering(
    nb_samples=15, # divide en 15 puntos las trayecorias
    features=["longitude", "latitude", "track_unwrapped"], # datos cebados al algorimto de ML
    clustering=KMeans(
                     n_clusters = 14,
                     n_init = 10,
                     random_state = 7), # algoritmo de ML utilizado, con los parámetros del mismo
    transform=StandardScaler(), # lo que hace esto es escalar los datos dentro de una distribuciñ¡ón Gausiana
).fit_predict()

In [ ]:
representarcluster(vuelos_kmeans,"cluster_kmeans.png",True)